In [2]:
Today = '31.12.2023'

Data_file = 'sapdeals31122023' + '.xlsx'
Sheet_in_data_file = 'Export Worksheet'

from datetime import date

Output_file = "_".join([str(date.today()),'EX_SUEK_Deals.xlsx'])
Sheet_in_output_file = 'Deals'

In [2]:
import cx_Oracle

ModuleNotFoundError: No module named 'cx_Oracle'

In [3]:
# !dir

In [4]:
import pandas as pd
import numpy as np

data_export = pd.read_excel(Data_file, sheet_name = Sheet_in_data_file)

In [5]:
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\OCP")
from Defs import merge_SalesUnits
from Defs import merge_Mapping

In [58]:
day = pd.to_datetime(Today)
data_work = data_export

data_work = data_work.query("dealStatusName != 'Cancelled' & invoiced == 'Y' & deliveryPeriodTo > @day").reset_index(drop=True)

data_work[['sellerName', 'buyerName']] = data_work[['sellerName', 'buyerName']].fillna('')
data_work['Counterparty'] = data_work['sellerName'] + data_work['buyerName']

data_work.loc[data_work['actualQuantity'].isna(), 'actualQuantity'] = data_work.loc[data_work['actualQuantity'].isna(), 'BLTonnage']

data_work['Sum_money'] = data_work['actualQuantity'] * data_work['conditionPrice']

data_work.loc[data_work.dealTypeName == 'Purchasing Deal', 'actualQuantity'] = data_work.actualQuantity.abs()
data_work.loc[data_work.dealTypeName == 'Purchasing Deal', 'Sum_money'] = -1 * data_work.Sum_money.abs()

data_work.loc[data_work.dealTypeName == 'Sales Deal', 'actualQuantity'] = -1 * data_work.actualQuantity.abs()
data_work.loc[data_work.dealTypeName == 'Sales Deal', 'Sum_money'] = data_work.Sum_money.abs()

data_work['CompCode'] = merge_Mapping(data_work, col='companyName')
data_work['Business_segment'] = merge_SalesUnits(data_work, col='CompCode', merge_col='ocpSegment')

data_work['Cpty_Code'] = merge_Mapping(data_work, col='Counterparty')
data_work['Cpty_segment'] = merge_SalesUnits(data_work, col='CompCode', merge_col='Cpty_Code').fillna('External')

data_work['Days'] = 0
data_work['Days'] = data_work['deliveryPeriodTo'] - day
data_work['Days'] = data_work['Days'].dt.days
data_work['Period'] = '2Y+'
data_work.loc[pd.to_numeric(data_work['Days']) < 730, 'Period'] = '1Y-2Y'
data_work.loc[pd.to_numeric(data_work['Days']) < 365, 'Period'] = '6M-1Y'
data_work.loc[pd.to_numeric(data_work['Days']) < 182, 'Period'] = '3M-6M'
data_work.loc[pd.to_numeric(data_work['Days']) < 91, 'Period'] = '1M-3M'
data_work.loc[pd.to_numeric(data_work['Days']) < 30, 'Period'] = '<1M'

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_11672\3507667553.py:1: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  day = pd.to_datetime(Today)


In [59]:
data_for_print = data_work[['productTypeName', 'companyName', 'Business_segment', 'Counterparty',  'Cpty_segment', 'currency', 'actualQuantity', 'Sum_money', 'deliveryPeriodTo', 'Days', 'Period']]
data_for_print.to_excel(Output_file, sheet_name=Sheet_in_output_file, index=False)